<a href="https://colab.research.google.com/github/code-deenah/AI-Physics-Tutor/blob/main/LLMs_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dolly 3B Model - by Databricks
---
1. Model Name: Dolly-v2-3b
2. Model Parameters: 3 Billion
3. Training: Instruction-tuned Model
4. Link: https://huggingface.co/databricks/dolly-v2-3b
---

In [ ]:
# install dependencies
!pip install transformers
!pip install sentencepiece
!pip install accelerate

In [ ]:
import torch                        # allows Tensor computation with strong GPU acceleration
from transformers import pipeline   # fast way to use pre-trained models for inference
import os

In [ ]:
# load model
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                            torch_dtype=torch.bfloat16,
                            trust_remote_code=True,
                            device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# define helper function
def get_completion_dolly(input):
  system = f"""
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  """
  prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from Dolly-v2-3b:"
  print(prompt)
  dolly_response = dolly_pipeline(prompt,
                                  max_new_tokens=500
                                  )
  return dolly_response[0]["generated_text"]

In [ ]:
# let's prompt
prompt = "Explain the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"

print(get_completion_dolly(prompt))

#### System: 
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  
#### User: 
Explain the difference between nuclear fission and fusion.

#### Response from Dolly-v2-3b:


### Build AI Physics Tutor

In [ ]:
!pip install gradio

In [ ]:
import locale
# print(locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"

# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

In [ ]:
import gradio as gr

# build an app front-end / ai tool with Gradio
iface = gr.Interface(fn=get_completion_dolly, inputs=[gr.Textbox(label="Insert Prompt Here",lines=6)],
                     outputs=[gr.Textbox(label="Your Answer Here",lines=3)],
                     title="My AI Physics Teacher",
                     examples=["Explain the difference between nuclear fusion and fission.",
                              "Why is the Sky blue?"]
                     )
iface.launch(share=True)

In [ ]:
iface.close()

## Falcon 7B Model - by TII
---
1. Model Name: Falcon-7b-instruct
2. Model Parameters: 7 Billion
3. Training: Instruction-tuned Model
4. Link: https://huggingface.co/tiiuae/falcon-7b-instruct
---

In [ ]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

In [ ]:
# define completion function
def get_completion_falcon(input):
  # prompt = f"You are an expert Physicist. Your job is to explain complex Physics concepts in simple words. \n{input}"
  # print(prompt)
  system = f"""
  You are an expert Physicist.
  You are good at explaining Physics concepts in simple words.
  Help as much as you can.
  """
  prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
  print(prompt)
  falcon_response = falcon_pipeline(prompt,
                                    max_length=500,
                                    do_sample=True,
                                    top_k=10,
                                    num_return_sequences=1,
                                    eos_token_id=tokenizer.eos_token_id,
                                    )

  return falcon_response

In [ ]:
# let's prompt
prompt = "Explain to me the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"
response = get_completion_falcon(prompt)
print(response[0]['generated_text'])

In [ ]:
print(response[0]['generated_text'])

## Stable Diffusion v1-5 - by RunwayML
---
1. Model Name: stable-diffusion-v1-5
2. Model type: Diffusion-based text-to-image generation model
3. Model Description: Generate/ modify images w/ text prompts.
4. Link: https://huggingface.co/runwayml/stable-diffusion-v1-5
---

In [ ]:
# Developed by: Robin Rombach, Patrick Esser
# Latent Diffusion Model that uses a fixed, pretrained text encoder (CLIP ViT-L/14) as suggested in the Imagen paper.
# Language(s): English
# License: The CreativeML OpenRAIL M license is an Open RAIL M license, adapted from the work that
# BigScience and the RAIL Initiative are jointly carrying in the area of responsible AI licensing.
# See also the article about the BLOOM Open RAIL license on which our license is based.
# https://bigscience.huggingface.co/blog/the-bigscience-rail-license

In [ ]:
# install dependencies
!pip install diffusers
!pip install transformers

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

In [ ]:
def get_completion_sd(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

In [ ]:
#let's prompt

# prompt = "astronaut, riding a horse, on mars, human colony"
# prompt = "children, playing in disneyland, view from a distance"
prompt = """baby llama, wearing red muffler,
grazing, open field, sunset
"""
print(prompt)
sd_image = get_completion_sd(prompt)
sd_image.save("./llama.jpg")

### Build AI Image Generator

In [ ]:
!pip install gradio==3.48.0

In [ ]:
import gradio as gr

In [ ]:
def get_completion(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

# def generate(prompt):
#     output = get_completion_sd(prompt)
#     return output

genai_app = gr.Interface(fn=get_completion,
                         inputs=[gr.Textbox(label="Your prompt")],
                         outputs=[gr.Image(label="Result")],
                         title="Generate Cool Images",
                         description="Generate any image with Stable Diffusion",
                         allow_flagging="never",
                         examples=["astronaut, riding a horse, on mars",
                                   "cargo ship, flying, in space"])
genai_app.launch()

In [ ]:
# gr.close_all()